# Chapter 9 - Temporal forcing and time-dependent parameters 

```{contents} Table of Contents
:depth: 3
```


Up until this point, we have assumed that the dynamical systems governing the spread of a pathogen rely on a finite set of fixed parameter values. 
For example, for the SIR model, we assume that $\beta$ the transmission rate and $1/\gamma$ the average infectious period do not change over time. 

\begin{align}
    \frac{dS}{dt} &= -\beta S \frac{I}{N}\\ 
    \frac{dI}{dt} &= \beta S \frac{I}{N} - \gamma I  \\ 
    \frac{dR}{dt} &= \gamma I
\end{align}

We see that the parameter values in the above model are constants (i.e. not functions of time). 

There are many different situations where time-dependent parameters are a more appropriate way to model an outbreak.

## Intervention for compartmental models
The simplest example is when we assume that transmission has changed because of an intervention.
This was explored in Chapter 4 for the Reed-Frost model. 
For completeness we will explore an intervention model below for compartmental models. 

Assume that we wish to model the number of susceptible, infectious, and removed individuals from time $0$ to time $T$ in increments of $1$. For example, we may assume that we observe these diseases states from week 0 to week 31 in increments of one week. 

Assume that an intervention will be implemented at time $T'$.
When the intervention is implemented then it will reduce the transmission rate by a proportion $\rho$ for $0 < \rho < 1$ for $\tau$ time units. 

The above assumptions change our SIR model from one with a constant set of parameters $\theta = (\beta, \gamma)$ to a model with parameter values that depend on time. These models are (often) called **Time-dependent models** or sometimes a model with time-varying parameters, etc. 

We modify the above SIR such that the constant $\beta$ is not a function of time unit, $t$, or \beta(t).
Note: Time-dependent models include models with constant parameter values. 